In [1]:
import malaya

In [2]:
bert = malaya.sentiment.transformer(model = 'bert')
xlnet = malaya.sentiment.transformer(model = 'xlnet')
albert = malaya.sentiment.transformer(model = 'albert')

In [3]:
import numpy as np
import json

In [4]:
with open('ps.json') as fopen:
    ps = json.load(fopen)
    
len(ps)

1531001

In [5]:
from tqdm import tqdm

batch_size = 128

results = []
for i in tqdm(range(0, len(ps), batch_size)):
    index = min(i + batch_size, len(ps))
    batch = ps[i: index]
    bert_r = bert.predict_batch(batch,get_proba = True,add_neutral=False)
    xlnet_r = xlnet.predict_batch(batch,get_proba = True,add_neutral=False)
    albert_r = albert.predict_batch(batch,get_proba = True,add_neutral=False)
    bert_r = np.array([list(r.values()) for r in bert_r])
    xlnet_r = np.array([list(r.values()) for r in xlnet_r])
    albert_r = np.array([list(r.values()) for r in albert_r])
    results.append((bert_r + xlnet_r + albert_r) / 3)

 74%|███████▎  | 8807/11961 [1:22:48<40:17,  1.30it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 11961/11961 [2:07:22<00:00,  1.57it/s]


In [6]:
results = np.concatenate(results, axis = 0)
results.shape

(1531001, 2)

In [7]:
Y = [1] * (results.shape[0] - 1) + [0]

In [8]:
numpy_array_of_noisy_labels = np.array(Y)
results.shape, numpy_array_of_noisy_labels.shape 

((1531001, 2), (1531001,))

In [9]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s = numpy_array_of_noisy_labels,
    psx = results,
    sorted_index_method='normalized_margin',
 )

In [10]:
ordered_label_errors.shape

(445282,)

In [12]:
for i in ordered_label_errors[:1000]:
    if np.argmax(results[i]) == 0:
        print(results[i][0], ps[i].replace('\n', '. '))

0.9999213 @Intan_Hariza BODOHLAH KAU SUARA DAHLAH MACAM LANGAU NYANYI NASIB BAIK MENANG NO 2 
0.999921 Jalan jalan ke kota batik. Dianterin sama si uci. Eh neng jangan so cantik. Muka lu kaya pantat panci. . Apeloo!!
0.99991924 Dekat temerloh ni semua benda nak mahal tapi takda benda yang maju pon lancau
0.9999191 Pandai dia gi Penang ON GRINDR PALA BUTOH PINJAM DUIT AKU X BAGI BALIK LAK TU  KESIAN MAK AKU BODO XDE CUCU KADHAKSNSBSB BABII https://t.co/17K5jmqsp4
0.9999182 Lancau lah nama dato sri bagai pun mintak sedekah ke? Kau pun bingai boleh percaya..
0.9999175 @zafri_saiful ALAH BABILAH BAGI JE LAH !!!! AKU BUKAN NAK KAWIN 4 PUN NAK ADIL2 BAGAI ANJIR
0.9999172 @_hafizulasnan @zarulizhm lancau aku D.E last sem baru lulus jgn cerita please duka lara. math2 tu pun smpai kena p https://t.co/JrzinZs5K1
0.9999166 @CHickENgorenk @alvnnr UNGA BABINYA, APING JAGA LILIN, AKU AMBIL UANGNYA. JOBDESC YG ADIL
0.9999164 @kdaniesl @younghoonw Kenapa sih qm sensi binggo kaya pantat bayi :(
0.99991

In [13]:
not_in = set(np.arange(len(ps))) - set(ordered_label_errors)

strong_positives = []
for i in tqdm(not_in):
    strong_positives.append(ps[i])

100%|██████████| 1085719/1085719 [00:00<00:00, 1914578.12it/s]


In [14]:
len(strong_positives)

1085719

In [15]:
with open('strong_positives.json', 'w') as fopen:
    json.dump(strong_positives, fopen)

In [17]:
weak_negatives = []
for i in tqdm(ordered_label_errors):
    weak_negatives.append(ps[i])

100%|██████████| 445282/445282 [00:00<00:00, 1278386.19it/s]


In [18]:
with open('weak-negatives.json', 'w') as fopen:
    json.dump(weak_negatives, fopen)